In [ ]:
import string

# Символи пунктуації
punct = set(string.punctuation)

# Морфологічні правила, що використовуються для класифікації невідомих слів
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


In [ ]:
def assign_unk(tok):
    """
    Призначення міток для невідомих слів
    """
    # Цифри
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Пунктуація
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Великі літери
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Іменники
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Дієслова
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Прикметники
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Прислівники
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"


In [ ]:
def preprocess(vocab, data_fp):
    """
    Попередня обробка даних
    """
    orig = []
    prep = []

    # Додаю спеціальний токен для кінця речення
    if "--n--" not in vocab:
        vocab["--n--"] = len(vocab)

    # Читання даних
    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # Кінець речення
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Обробка невідомих слів
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word.strip())  # Виправлення: додано strip()
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [ ]:
import nltk
import random
from collections import Counter

In [ ]:
nltk.download('conll2000')
from nltk.corpus import conll2000

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


In [ ]:
tagged_sents = list(conll2000.tagged_sents())

In [ ]:
random.seed(42)
random.shuffle(tagged_sents)

In [ ]:
split_point = int(len(tagged_sents) * 0.8)
train_sents = tagged_sents[:split_point]
test_sents = tagged_sents[split_point:]

In [ ]:
def write_tagged_sentences(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for word, tag in sentence:
                f.write(f"{word}\t{tag}\n")
            f.write("\n")

In [ ]:
write_tagged_sentences(train_sents, "conll2000_training.pos")
write_tagged_sentences(test_sents, "conll2000_test.pos")

In [ ]:
def get_count(word_l):
    """
    Створює словник частотності слів.
    """
    word_count_dict = {}
    for w in word_l:
        if w in word_count_dict:
            word_count_dict[w] += 1
        else:
            word_count_dict[w] = 1
    return word_count_dict

In [ ]:
c_vocab_indexed = {word: idx for idx, word in enumerate(c_vocab.keys())}

In [ ]:
c_vocab_indexed

{'PNC': 0,
 'waves': 1,
 'refinancing': 2,
 'outlook': 3,
 'Gen.': 4,
 'stress': 5,
 'sports': 6,
 'bogged': 7,
 'growth': 8,
 'sheer': 9,
 'shouted': 10,
 'noncriminal': 11,
 'serving': 12,
 '2683.20': 13,
 'ranging': 14,
 'worry': 15,
 'kickbacks': 16,
 'Dillow': 17,
 'rapidly': 18,
 '165': 19,
 'general': 20,
 'maitre': 21,
 'kicked': 22,
 'breaking': 23,
 'subsequently': 24,
 'falls': 25,
 'avoided': 26,
 'mixed': 27,
 'reliable': 28,
 'Brierley': 29,
 'President': 30,
 'expansion': 31,
 'load': 32,
 'compact': 33,
 'breath': 34,
 'THE': 35,
 'reviewing': 36,
 'minor': 37,
 'irrational': 38,
 'Pension': 39,
 'unexpected': 40,
 'Newport': 41,
 'Gras': 42,
 'outlawed': 43,
 'handle': 44,
 'Florio': 45,
 'pause': 46,
 'totally': 47,
 'Penn': 48,
 'deaths': 49,
 'career': 50,
 'Often': 51,
 'N.C.': 52,
 "'ll": 53,
 'scenes': 54,
 'Vegas': 55,
 'undermined': 56,
 'landslide': 57,
 'NatWest': 58,
 'Rio': 59,
 'repurchase': 60,
 'suggest': 61,
 'hospitalized': 62,
 '1.44': 63,
 'Funds': 6

In [ ]:
c_vocab = get_count(vocab)

In [ ]:
c_vocab

{'PNC': 1,
 'waves': 1,
 'refinancing': 1,
 'outlook': 1,
 'Gen.': 1,
 'stress': 1,
 'sports': 1,
 'bogged': 1,
 'growth': 1,
 'sheer': 1,
 'shouted': 1,
 'noncriminal': 1,
 'serving': 1,
 '2683.20': 1,
 'ranging': 1,
 'worry': 1,
 'kickbacks': 1,
 'Dillow': 1,
 'rapidly': 1,
 '165': 1,
 'general': 1,
 'maitre': 1,
 'kicked': 1,
 'breaking': 1,
 'subsequently': 1,
 'falls': 1,
 'avoided': 1,
 'mixed': 1,
 'reliable': 1,
 'Brierley': 1,
 'President': 1,
 'expansion': 1,
 'load': 1,
 'compact': 1,
 'breath': 1,
 'THE': 1,
 'reviewing': 1,
 'minor': 1,
 'irrational': 1,
 'Pension': 1,
 'unexpected': 1,
 'Newport': 1,
 'Gras': 1,
 'outlawed': 1,
 'handle': 1,
 'Florio': 1,
 'pause': 1,
 'totally': 1,
 'Penn': 1,
 'deaths': 1,
 'career': 1,
 'Often': 1,
 'N.C.': 1,
 "'ll": 1,
 'scenes': 1,
 'Vegas': 1,
 'undermined': 1,
 'landslide': 1,
 'NatWest': 1,
 'Rio': 1,
 'repurchase': 1,
 'suggest': 1,
 'hospitalized': 1,
 '1.44': 1,
 'Funds': 1,
 'exporting': 1,
 'National': 1,
 'Freeway': 1,
 'Mi

In [ ]:
word_counter = Counter()
for sentence in train_sents:
    for word, _ in sentence:
        word_counter[word] += 1

In [ ]:
vocab = {word for word, count in word_counter.items() if count >= 2}

In [ ]:
vocab

{'PNC',
 'waves',
 'refinancing',
 'outlook',
 'Gen.',
 'stress',
 'sports',
 'bogged',
 'growth',
 'sheer',
 'shouted',
 'noncriminal',
 'serving',
 '2683.20',
 'ranging',
 'worry',
 'kickbacks',
 'Dillow',
 'rapidly',
 '165',
 'general',
 'maitre',
 'kicked',
 'breaking',
 'subsequently',
 'falls',
 'avoided',
 'mixed',
 'reliable',
 'Brierley',
 'President',
 'expansion',
 'load',
 'compact',
 'breath',
 'THE',
 'reviewing',
 'minor',
 'irrational',
 'Pension',
 'unexpected',
 'Newport',
 'Gras',
 'outlawed',
 'handle',
 'Florio',
 'pause',
 'totally',
 'Penn',
 'deaths',
 'career',
 'Often',
 'N.C.',
 "'ll",
 'scenes',
 'Vegas',
 'undermined',
 'landslide',
 'NatWest',
 'Rio',
 'repurchase',
 'suggest',
 'hospitalized',
 '1.44',
 'Funds',
 'exporting',
 'National',
 'Freeway',
 'Missouri',
 'farm',
 'insider',
 'Profits',
 'confessed',
 'thin',
 'USAir',
 'members',
 'millionaires',
 'firms',
 'two-income',
 'Block',
 'plunging',
 'passed',
 'Weirton',
 'advocates',
 'Nobel',
 'Pol

In [ ]:
with open("conll2000_vocab.txt", 'w', encoding='utf-8') as f:
    for word in sorted(vocab):
        f.write(f"{word}\n")

In [ ]:
with open("conll2000_test_words.txt", 'w', encoding='utf-8') as f:
    for sentence in test_sents:
        for word, _ in sentence:
            f.write(f"{word}\n")
        f.write("\n")

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import math


In [ ]:
with open("/root/nltk_data/corpora/conll2000/train.txt", 'r') as f:
    training_corpus = f.readlines()

In [ ]:
training_corpus

['Confidence NN B-NP\n',
 'in IN B-PP\n',
 'the DT B-NP\n',
 'pound NN I-NP\n',
 'is VBZ B-VP\n',
 'widely RB I-VP\n',
 'expected VBN I-VP\n',
 'to TO I-VP\n',
 'take VB I-VP\n',
 'another DT B-NP\n',
 'sharp JJ I-NP\n',
 'dive NN I-NP\n',
 'if IN B-SBAR\n',
 'trade NN B-NP\n',
 'figures NNS I-NP\n',
 'for IN B-PP\n',
 'September NNP B-NP\n',
 ', , O\n',
 'due JJ B-ADJP\n',
 'for IN B-PP\n',
 'release NN B-NP\n',
 'tomorrow NN B-NP\n',
 ', , O\n',
 'fail VB B-VP\n',
 'to TO I-VP\n',
 'show VB I-VP\n',
 'a DT B-NP\n',
 'substantial JJ I-NP\n',
 'improvement NN I-NP\n',
 'from IN B-PP\n',
 'July NNP B-NP\n',
 'and CC I-NP\n',
 'August NNP I-NP\n',
 "'s POS B-NP\n",
 'near-record JJ I-NP\n',
 'deficits NNS I-NP\n',
 '. . O\n',
 '\n',
 'Chancellor NNP O\n',
 'of IN B-PP\n',
 'the DT B-NP\n',
 'Exchequer NNP I-NP\n',
 'Nigel NNP B-NP\n',
 'Lawson NNP I-NP\n',
 "'s POS B-NP\n",
 'restated VBN I-NP\n',
 'commitment NN I-NP\n',
 'to TO B-PP\n',
 'a DT B-NP\n',
 'firm NN I-NP\n',
 'monetary JJ 

In [ ]:
with open("/root/nltk_data/corpora/conll2000/corpus.txt", 'r') as f:
    voc_l = f.read().split('\n')


In [ ]:
voc_l

['Newsletter of the EMS – Manuscript',
 '',
 'Page 1',
 '',
 'The mathematics of Bitcoin',
 'Cyril Grunspan (De Vinci Research Center, Paris, France)',
 'Ricardo Pérez-Marco (CNRS, IMJ-PRG, Sorbonne Université, Paris, France)',
 '',
 'arXiv:2003.00001v1 [math.HO] 2 Mar 2020',
 '',
 '1',
 '',
 'Introduction to Bitcoin.',
 '',
 'Bitcoin is a new decentralized payment network that started',
 'operating in January 2009. This new technology was created',
 'by a pseudonymous author, or group of authors, called Satoshi',
 'Nakamoto in an article that was publically released [1] in the',
 'cypherpunk mailing list. The cypherpunks are anarchists and',
 'cryptographers that who have been concerned with personal',
 'privacy in the Internet since the 90’s. This article follows on',
 'a general presentation of Bitcoin by the second author [2].',
 'We refer to this previous article for general background. Here',
 'we focuss on mathematics being a feature of the security and',
 'effectiveness of Bitc

In [ ]:
vocab = {}
for i, word in enumerate(sorted(voc_l)):
    vocab[word] = i

1257933

In [ ]:
with open("/root/nltk_data/corpora/conll2000/test.txt", 'r') as f:
    y = f.readlines()

In [ ]:
y

['Rockwell NNP B-NP\n',
 'International NNP I-NP\n',
 'Corp. NNP I-NP\n',
 "'s POS B-NP\n",
 'Tulsa NNP I-NP\n',
 'unit NN I-NP\n',
 'said VBD B-VP\n',
 'it PRP B-NP\n',
 'signed VBD B-VP\n',
 'a DT B-NP\n',
 'tentative JJ I-NP\n',
 'agreement NN I-NP\n',
 'extending VBG B-VP\n',
 'its PRP$ B-NP\n',
 'contract NN I-NP\n',
 'with IN B-PP\n',
 'Boeing NNP B-NP\n',
 'Co. NNP I-NP\n',
 'to TO B-VP\n',
 'provide VB I-VP\n',
 'structural JJ B-NP\n',
 'parts NNS I-NP\n',
 'for IN B-PP\n',
 'Boeing NNP B-NP\n',
 "'s POS B-NP\n",
 '747 CD I-NP\n',
 'jetliners NNS I-NP\n',
 '. . O\n',
 '\n',
 'Rockwell NNP B-NP\n',
 'said VBD B-VP\n',
 'the DT B-NP\n',
 'agreement NN I-NP\n',
 'calls VBZ B-VP\n',
 'for IN B-SBAR\n',
 'it PRP B-NP\n',
 'to TO B-VP\n',
 'supply VB I-VP\n',
 '200 CD B-NP\n',
 'additional JJ I-NP\n',
 'so-called JJ I-NP\n',
 'shipsets NNS I-NP\n',
 'for IN B-PP\n',
 'the DT B-NP\n',
 'planes NNS I-NP\n',
 '. . O\n',
 '\n',
 'These DT B-NP\n',
 'include VBP B-VP\n',
 ', , O\n',
 'amo

In [ ]:
_, prep = preprocess(vocab, "/root/nltk_data/corpora/conll2000/test.txt")

In [ ]:
_

['Rockwell NNP B-NP',
 'International NNP I-NP',
 'Corp. NNP I-NP',
 "'s POS B-NP",
 'Tulsa NNP I-NP',
 'unit NN I-NP',
 'said VBD B-VP',
 'it PRP B-NP',
 'signed VBD B-VP',
 'a DT B-NP',
 'tentative JJ I-NP',
 'agreement NN I-NP',
 'extending VBG B-VP',
 'its PRP$ B-NP',
 'contract NN I-NP',
 'with IN B-PP',
 'Boeing NNP B-NP',
 'Co. NNP I-NP',
 'to TO B-VP',
 'provide VB I-VP',
 'structural JJ B-NP',
 'parts NNS I-NP',
 'for IN B-PP',
 'Boeing NNP B-NP',
 "'s POS B-NP",
 '747 CD I-NP',
 'jetliners NNS I-NP',
 '. . O',
 '',
 'Rockwell NNP B-NP',
 'said VBD B-VP',
 'the DT B-NP',
 'agreement NN I-NP',
 'calls VBZ B-VP',
 'for IN B-SBAR',
 'it PRP B-NP',
 'to TO B-VP',
 'supply VB I-VP',
 '200 CD B-NP',
 'additional JJ I-NP',
 'so-called JJ I-NP',
 'shipsets NNS I-NP',
 'for IN B-PP',
 'the DT B-NP',
 'planes NNS I-NP',
 '. . O',
 '',
 'These DT B-NP',
 'include VBP B-VP',
 ', , O',
 'among IN B-PP',
 'other JJ B-NP',
 'parts NNS I-NP',
 ', , O',
 'each DT B-NP',
 'jetliner NN I-NP',
 "

In [ ]:
def get_word_tag(line, vocab):
    """
    Отримання слова та його тегу з рядка корпусу.
    """
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag  # Виправлення: додано повернення значення
    else:
        parts = line.split()
        if len(parts) >= 2:
            word, tag = parts[0], parts[1]
            if word not in vocab:
                # Обробка невідомих слів
                word = assign_unk(word)
            return word, tag
        else:
            # Обробка некоректних рядків
            return "--n--", "--s--"


In [ ]:
get_word_tag('duck','duck')

('--n--', '--s--')

In [ ]:
def create_dictionaries(training_corpus, vocab):
    """
    Створення словників частот.
    """
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    # Початковий тег
    prev_tag = '--s--'

    for word_tag in training_corpus:
        word, tag = get_word_tag(word_tag, vocab)

        # Збільшення лічильника переходів
        transition_counts[(prev_tag, tag)] += 1

        # Збільшення лічильника емісій
        emission_counts[(tag, word)] += 1

        # Збільшення лічильника тегів
        tag_counts[tag] += 1

        # Оновлення попереднього тегу
        prev_tag = tag

    return emission_counts, transition_counts, tag_counts


In [ ]:
ec, trc, tagc = create_dictionaries(training_corpus, 'duck')

In [ ]:
ec

defaultdict(int,
            {('NN', '--unk_upper--'): 529,
             ('IN', '--unk--'): 18996,
             ('DT', '--unk--'): 15947,
             ('NN', '--unk--'): 18710,
             ('VBZ', '--unk--'): 4331,
             ('RB', '--unk_adj--'): 1835,
             ('VBN', '--unk--'): 4697,
             ('TO', '--unk--'): 5046,
             ('VB', '--unk--'): 5119,
             ('DT', '--unk_noun--'): 151,
             ('JJ', '--unk--'): 7497,
             ('NN', '--unk_adj--'): 299,
             ('NNS', '--unk--'): 12884,
             ('IN', '--unk_noun--'): 2450,
             ('NNP', '--unk_upper--'): 16859,
             (',', '--unk_punct--'): 10770,
             ('NN', '--unk_noun--'): 8233,
             ('CC', '--unk--'): 3966,
             ('POS', '--unk_punct--'): 1769,
             ('JJ', '--unk_punct--'): 1606,
             ('.', '--unk_punct--'): 8827,
             ('--s--', '--n--'): 8936,
             ('JJ', '--unk_noun--'): 970,
             ('CC', '--unk_upper--'): 5

In [ ]:
trc

defaultdict(int,
            {('--s--', 'NN'): 404,
             ('NN', 'IN'): 7468,
             ('IN', 'DT'): 7425,
             ('DT', 'NN'): 8884,
             ('NN', 'VBZ'): 1169,
             ('VBZ', 'RB'): 625,
             ('RB', 'VBN'): 570,
             ('VBN', 'TO'): 500,
             ('TO', 'VB'): 2966,
             ('VB', 'DT'): 1269,
             ('DT', 'JJ'): 3647,
             ('JJ', 'NN'): 5993,
             ('IN', 'NN'): 2539,
             ('NN', 'NNS'): 2561,
             ('NNS', 'IN'): 3153,
             ('IN', 'NNP'): 3112,
             ('NNP', ','): 2771,
             (',', 'JJ'): 420,
             ('JJ', 'IN'): 703,
             ('NN', 'NN'): 3546,
             ('NN', ','): 3391,
             (',', 'VB'): 49,
             ('VB', 'TO'): 283,
             ('NNP', 'CC'): 838,
             ('CC', 'NNP'): 865,
             ('NNP', 'POS'): 940,
             ('POS', 'JJ'): 338,
             ('JJ', 'NNS'): 2945,
             ('NNS', '.'): 1802,
             ('.', '--s--'

In [ ]:
tagc

defaultdict(int,
            {'NN': 30147,
             'IN': 22764,
             'DT': 18335,
             'VBZ': 4648,
             'RB': 6607,
             'VBN': 4763,
             'TO': 5081,
             'VB': 6017,
             'JJ': 13085,
             'NNS': 13619,
             'NNP': 19884,
             ',': 10770,
             'CC': 5372,
             'POS': 1769,
             '.': 8827,
             '--s--': 8936,
             'VBP': 2868,
             'VBG': 3272,
             'PRP$': 1881,
             'CD': 8315,
             '``': 1531,
             "''": 1493,
             'VBD': 6745,
             'EX': 206,
             'MD': 2167,
             '#': 36,
             '(': 274,
             '$': 1750,
             ')': 281,
             'NNPS': 420,
             'PRP': 3820,
             'JJS': 374,
             'WP': 529,
             'RBR': 321,
             'JJR': 853,
             'WDT': 955,
             'WRB': 478,
             'RBS': 191,
             'PDT': 55,

In [ ]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    """
    Створення матриці переходів A.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)

    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        for j in range(num_tags):
            key = (all_tags[i], all_tags[j])

            count = 0
            if key in transition_counts:
                count = transition_counts[key]

            count_prev_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)

    return A


In [ ]:
A = create_transition_matrix(1, tagc, trc)

In [ ]:
A

array([[0.01234568, 0.01234568, 0.01234568, ..., 0.01234568, 0.01234568,
        0.01234568],
       [0.0005571 , 0.0005571 , 0.0005571 , ..., 0.0005571 , 0.0005571 ,
        0.0005571 ],
       [0.0006502 , 0.00130039, 0.00780234, ..., 0.0006502 , 0.00520156,
        0.00780234],
       ...,
       [0.0125    , 0.0125    , 0.0125    , ..., 0.0125    , 0.0125    ,
        0.0125    ],
       [0.00191205, 0.00191205, 0.00191205, ..., 0.00191205, 0.00191205,
        0.00191205],
       [0.00063452, 0.00063452, 0.00063452, ..., 0.00063452, 0.01015228,
        0.00063452]])

In [ ]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    """
    Створення матриці емісій B.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocab)

    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        for j in range(num_words):
            key = (all_tags[i], vocab[j])

            count = 0
            if key in emission_counts:
                count = emission_counts[key]

            count_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)

    return B

In [ ]:
B = create_emission_matrix(1,tagc,ec,"duck")

In [ ]:
B

array([[2.50000000e-02, 2.50000000e-02, 2.50000000e-02, 2.50000000e-02],
       [5.70125428e-04, 5.70125428e-04, 5.70125428e-04, 5.70125428e-04],
       [6.68002672e-04, 6.68002672e-04, 6.68002672e-04, 6.68002672e-04],
       [3.59712230e-03, 3.59712230e-03, 3.59712230e-03, 3.59712230e-03],
       [3.50877193e-03, 3.50877193e-03, 3.50877193e-03, 3.50877193e-03],
       [9.28160386e-05, 9.28160386e-05, 9.28160386e-05, 9.28160386e-05],
       [1.11856823e-04, 1.11856823e-04, 1.11856823e-04, 1.11856823e-04],
       [1.13237459e-04, 1.13237459e-04, 1.13237459e-04, 1.13237459e-04],
       [9.51474786e-04, 9.51474786e-04, 9.51474786e-04, 9.51474786e-04],
       [1.86011905e-04, 1.86011905e-04, 1.86011905e-04, 1.86011905e-04],
       [1.20206756e-04, 1.20206756e-04, 1.20206756e-04, 1.20206756e-04],
       [5.45286003e-05, 5.45286003e-05, 5.45286003e-05, 5.45286003e-05],
       [4.76190476e-03, 4.76190476e-03, 4.76190476e-03, 4.76190476e-03],
       [2.38095238e-02, 2.38095238e-02, 2.38095238e

In [ ]:
B = create_emission_matrix(1,tagc,ec,vocab)

KeyError: 0

In [ ]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    """
    Ініціалізація алгоритму Вітербі.
    """
    num_tags = len(tag_counts)

    # Ініціалізація матриць best_probs та best_paths
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)

    # Індекс початкового стану
    s_idx = states.index("--s--")

    # Заповнення першого стовпця best_probs
    word = corpus[0]
    word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів
    for i in range(num_tags):
      best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, word_idx])
    return best_probs, best_paths


In [ ]:
states = sorted(tagc.keys())
best_probs, best_paths = initialize(states,tagc,A,B,voc_l,c_vocab_indexed)

In [ ]:
def viterbi_forward(A, B, corpus, best_probs, best_paths, vocab):
    """
    Пряме проходження алгоритму Вітербі.
    """
    num_tags = best_probs.shape[0]

    for i in range(1, len(corpus)):
        word = corpus[i]
        word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів

        for j in range(num_tags):
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags):
                try:

                  prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, word_idx])

                  if prob > best_prob_i:
                      best_prob_i = prob
                      best_path_i = k
                except IndexError:
                  pass
            try:
              best_probs[j, i] = best_prob_i
              best_paths[j, i] = best_path_i
            except TypeError:
              pass

    return best_probs, best_paths


In [ ]:
viterbi_forward(A,B,voc_l, best_probs, best_paths, c_vocab_indexed)

KeyboardInterrupt: 

In [ ]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    """
    Зворотне проходження алгоритму Вітербі.
    """
    m = best_paths.shape[1]
    z = [None] * m
    pred = [None] * m

    # Знаходження найкращого тегу для останнього слова
    best_prob_for_last_word = float('-inf')

    for k in range(best_probs.shape[0]):
        if best_probs[k, m-1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, m-1]
            z[m-1] = k

    pred[m-1] = states[z[m-1]]

    # Зворотне проходження для знаходження найкращих тегів
    for i in range(m-2, -1, -1):
        z[i] = best_paths[z[i+1], i+1]
        pred[i] = states[z[i]]

    return pred


In [ ]:
pred = viterbi_backward(best_probs, best_paths,voc_l,states)

In [ ]:
def compute_accuracy(pred, y):
    """
    Обчислення точності моделі POS-тегування.
    """
    num_correct = 0
    total = 1

    for prediction, y_item in zip(pred, y):
        y_item = y_item.strip()
        word_tag_tuple = y_item.split('\t')

        if len(word_tag_tuple) != 2:
            continue

        word, tag = word_tag_tuple

        if tag == prediction:
            num_correct += 1

        total += 1

    return num_correct / total


In [ ]:
compute_accuracy(pred,y)

0.0

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import treebank

# Завантаження необхідних ресурсів
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')

# Приклад тексту
text = "The economy's temperature will be taken from several vantage points this week."

# Токенізація тексту
tokens = word_tokenize(text)

# POS-тегування з використанням NLTK
nltk_tags = nltk.pos_tag(tokens)

print("Результат POS-тегування NLTK:")
for word, tag in nltk_tags:
    print(f"{word}: {tag}")

# Оцінка точності NLTK POS-tagger на корпусі treebank
train_data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:]

# Навчання NLTK тегера
from nltk.tag import UnigramTagger, BigramTagger
unigram_tagger = UnigramTagger(train_data)
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)

# Оцінка точності
nltk_accuracy = bigram_tagger.evaluate(test_data)
print(f"Точність NLTK BigramTagger: {nltk_accuracy:.4f}")

# Порівняння з нашою реалізацією
print(f"Точність нашої реалізації: {accuracy:.4f}")
print(f"Різниця: {abs(accuracy - nltk_accuracy):.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Спроба номер 3

In [1]:
import string

# Символи пунктуації
punct = set(string.punctuation)

# Морфологічні правила, що використовуються для класифікації невідомих слів
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


In [2]:
def assign_unk(tok):
    """
    Призначення міток для невідомих слів
    """
    # Цифри
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Пунктуація
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Великі літери
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Іменники
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Дієслова
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Прикметники
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Прислівники
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"


In [3]:
def preprocess(vocab, data_fp):
    """
    Попередня обробка даних
    """
    orig = []
    prep = []

    # Додаю спеціальний токен для кінця речення
    if "--n--" not in vocab:
        vocab["--n--"] = len(vocab)

    # Читання даних
    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # Кінець речення
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Обробка невідомих слів
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word.strip())  # Виправлення: додано strip()
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [9]:
import nltk
import random
from collections import Counter

# Завантаження корпусу (приклад для Brown)
nltk.download('conll2000')
from nltk.corpus import conll2000

# Отримання тегованих речень
tagged_sents = list(conll2000.tagged_sents())

# Перемішування та розділення на навчальну і тестову вибірки
random.seed(42)
random.shuffle(tagged_sents)

split_point = int(len(tagged_sents) * 0.8)
train_sents = tagged_sents[:split_point]
test_sents = tagged_sents[split_point:]

# Запис у файли
def write_tagged_sentences(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for word, tag in sentence:
                f.write(f"{word}\t{tag}\n")
            f.write("\n")  # Порожній рядок між реченнями

write_tagged_sentences(train_sents, "conll_training.pos")
write_tagged_sentences(test_sents, "conll_test.pos")

# Створення словника - виправлена версія
word_counter = Counter()
for sentence in train_sents:
    for word, _ in sentence:
        word_counter[word] += 1

# Збереження слів, які з'являються принаймні двічі
vocab = {word for word, count in word_counter.items() if count >= 2}

with open("conll_vocab.txt", 'w', encoding='utf-8') as f:
    for word in sorted(vocab):
        f.write(f"{word}\n")

# Створення файлу тестових слів
with open("conll_test_words.txt", 'w', encoding='utf-8') as f:
    for sentence in test_sents:
        for word, _ in sentence:
            f.write(f"{word}\n")
        f.write("\n")  # Порожній рядок між реченнями

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


In [5]:
# Виправлення 2: Виправлена функція get_word_tag
def get_word_tag(line, vocab):
    """
    Отримання слова та його тегу з рядка корпусу.
    """
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag  # Виправлення: додано повернення значення
    else:
        parts = line.split()
        if len(parts) >= 2:
            word, tag = parts[0], parts[1]
            if word not in vocab:
                # Обробка невідомих слів
                word = assign_unk(word)
            return word, tag
        else:
            # Обробка некоректних рядків
            return "--n--", "--s--"  # Виправлення: додано повернення для некоректних рядків


In [15]:
def create_dictionaries(training_corpus, vocab):
    """
    Створення словників частот.
    """
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    # Початковий тег
    prev_tag = '--s--'

    for word_tag in training_corpus:
        word, tag = get_word_tag(word_tag, vocab)

        # Збільшення лічильника переходів
        transition_counts[(prev_tag, tag)] += 1

        # Збільшення лічильника емісій
        emission_counts[(tag, word)] += 1

        # Збільшення лічильника тегів
        tag_counts[tag] += 1

        # Оновлення попереднього тегу
        prev_tag = tag

    return emission_counts, transition_counts, tag_counts


In [24]:
import numpy as np
import pandas as pd
from collections import defaultdict
import math


In [35]:
with open("conll_training.pos", "r", encoding="utf-8") as f:
    training_corpus = f.readlines()

with open("conll_vocab.txt", "r", encoding="utf-8") as f:
    voc_list = f.read().split('\n')
vocab = {word: i for i, word in enumerate(sorted(voc_list)) if word}

emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [19]:
emission_counts

defaultdict(int,
            {('DT', 'No'): 15,
             ('NN', 'replacement'): 3,
             ('VBD', 'was'): 901,
             ('RB', 'immediately'): 14,
             ('VBN', 'named'): 30,
             ('.', '.'): 8518,
             ('--s--', '--n--'): 8758,
             ('NN', 'Revenue'): 46,
             ('VBD', 'declined'): 53,
             ('CD', '6.3'): 5,
             ('NN', '%'): 1163,
             ('TO', 'to'): 4987,
             ('$', '$'): 1664,
             ('CD', '1.11'): 4,
             ('CD', 'billion'): 371,
             ('IN', 'from'): 994,
             ('CD', '--unk_digit--'): 1210,
             ('``', '``'): 1462,
             ('JJ', 'Next'): 7,
             ('NN', 'year'): 468,
             ('VBZ', 'is'): 1360,
             ('DT', 'an'): 687,
             ('NN', 'election'): 11,
             ('CC', 'and'): 3503,
             ('DT', 'the'): 9041,
             ('NNS', 'legislators'): 9,
             ('RB', 'just'): 106,
             ('VBP', 'do'): 130,
         

In [20]:
transition_counts

defaultdict(int,
            {('--s--', 'DT'): 1880,
             ('DT', 'NN'): 8589,
             ('NN', 'VBD'): 1451,
             ('VBD', 'RB'): 515,
             ('RB', 'VBN'): 563,
             ('VBN', '.'): 215,
             ('.', '--s--'): 8062,
             ('--s--', 'NN'): 368,
             ('VBD', 'CD'): 487,
             ('CD', 'NN'): 1648,
             ('NN', 'TO'): 1233,
             ('TO', '$'): 300,
             ('$', 'CD'): 1657,
             ('CD', 'CD'): 1625,
             ('CD', 'IN'): 717,
             ('IN', '$'): 607,
             ('CD', '.'): 611,
             ('--s--', '``'): 625,
             ('``', 'JJ'): 146,
             ('JJ', 'NN'): 5911,
             ('NN', 'VBZ'): 1114,
             ('VBZ', 'DT'): 753,
             ('NN', 'NN'): 3458,
             ('NN', 'CC'): 1188,
             ('CC', 'DT'): 580,
             ('DT', 'NNS'): 1344,
             ('NNS', 'RB'): 409,
             ('RB', 'VBP'): 180,
             ('VBP', 'RB'): 403,
             ('RB', 'VB')

In [21]:
tag_counts

defaultdict(int,
            {'DT': 17939,
             'NN': 29396,
             'VBD': 6716,
             'RB': 6346,
             'VBN': 4700,
             '.': 8624,
             '--s--': 8758,
             'CD': 8180,
             'TO': 5021,
             '$': 1689,
             'IN': 22201,
             '``': 1489,
             'JJ': 12901,
             'VBZ': 4469,
             'CC': 5224,
             'NNS': 13231,
             'VBP': 2701,
             'VB': 5803,
             "''": 1451,
             'RBR': 311,
             'VBG': 3180,
             ',': 10566,
             'PRP$': 1819,
             'NNP': 19834,
             'PRP': 3694,
             'PDT': 47,
             'MD': 2085,
             'FW': 35,
             ':': 1066,
             'POS': 1769,
             'RBS': 189,
             'WRB': 462,
             'JJR': 841,
             'WP': 501,
             'WDT': 912,
             'JJS': 360,
             '(': 278,
             ')': 283,
             'RP': 75,
 

In [22]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    """
    Створення матриці переходів A.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)

    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        for j in range(num_tags):
            key = (all_tags[i], all_tags[j])

            count = 0
            if key in transition_counts:
                count = transition_counts[key]

            count_prev_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)

    return A

In [32]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    """
    Створення матриці емісій B.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocab)

    B = np.zeros((num_tags, num_words))

    index_to_word = {index: word for word, index in vocab.items()}

    for i in range(num_tags):
        for j in range(num_words):
            key = (all_tags[i], index_to_word[j])

            count = emission_counts.get(key, 0)

            count_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)

    return B

In [42]:
A = create_transition_matrix(0.1, tag_counts, transition_counts)

In [43]:
A

array([[2.10526316e-03, 2.10526316e-03, 2.10526316e-03, ...,
        2.10526316e-03, 2.10526316e-03, 2.10526316e-03],
       [5.90493062e-05, 5.90493062e-05, 5.90493062e-05, ...,
        5.90493062e-05, 5.90493062e-05, 5.90493062e-05],
       [6.87049124e-05, 7.55754036e-04, 6.25214703e-03, ...,
        6.87049124e-05, 4.19099966e-03, 6.25214703e-03],
       ...,
       [2.59740260e-03, 2.85714286e-02, 2.59740260e-03, ...,
        2.59740260e-03, 2.59740260e-03, 2.85714286e-02],
       [2.14362272e-04, 2.14362272e-04, 2.14362272e-04, ...,
        2.14362272e-04, 2.14362272e-04, 2.14362272e-04],
       [6.69568129e-05, 6.69568129e-05, 6.69568129e-05, ...,
        6.69568129e-05, 8.10177436e-03, 6.69568129e-05]])

In [41]:
B = create_emission_matrix(0.1, tag_counts, emission_counts, vocab)

In [40]:
vocab = {word: idx - 1 for word, idx in vocab.items()}

In [44]:
B

array([[9.90982063e-05, 4.27113269e-02, 9.90982063e-05, ...,
        9.90982063e-05, 9.90982063e-05, 9.90982063e-05],
       [3.76633648e-05, 3.76633648e-05, 6.26756054e-01, ...,
        3.76633648e-05, 3.76633648e-05, 3.76633648e-05],
       [4.13718919e-05, 4.13718919e-05, 4.13718919e-05, ...,
        4.13718919e-05, 4.13718919e-05, 4.13718919e-05],
       ...,
       [9.99900010e-05, 9.99900010e-05, 9.99900010e-05, ...,
        9.99900010e-05, 9.99900010e-05, 9.99900010e-05],
       [7.00231076e-05, 7.00231076e-05, 7.00231076e-05, ...,
        7.00231076e-05, 7.00231076e-05, 7.00231076e-05],
       [4.07315384e-05, 4.07315384e-05, 4.07315384e-05, ...,
        4.07315384e-05, 4.07315384e-05, 4.07315384e-05]])

In [46]:
# Виправлення 3: Оновлення функцій Вітербі для роботи з невідомими словами
def initialize(states, tag_counts, A, B, corpus, vocab):
    """
    Ініціалізація алгоритму Вітербі.
    """
    num_tags = len(tag_counts)

    # Ініціалізація матриць best_probs та best_paths
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)

    # Індекс початкового стану
    s_idx = states.index("--s--")

    # Заповнення першого стовпця best_probs
    word = corpus[0]
    word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів

    for i in range(num_tags):
        best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, word_idx])

    return best_probs, best_paths

In [48]:
states = sorted(tag_counts.keys())
best_probs, best_paths = initialize(states, tag_counts, A, B, training_corpus, vocab)

In [50]:
best_probs

array([[-20.60022084,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-17.85407225,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-17.54190484,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [-18.19336672,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-14.56569087,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-12.74883276,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [51]:
best_paths

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
def viterbi_forward(A, B, corpus, best_probs, best_paths, vocab):
    """
    Пряме проходження алгоритму Вітербі.
    """
    num_tags = best_probs.shape[0]

    for i in range(1, len(corpus)):
        word = corpus[i]
        word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів

        for j in range(num_tags):
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags):
                prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, word_idx])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k

            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i

    return best_probs, best_paths


In [54]:
best_probs, best_paths = viterbi_forward(A, B, training_corpus, best_probs, best_paths, vocab)

In [56]:
best_probs

array([[-2.06002208e+01, -3.05768190e+01, -4.26793891e+01, ...,
        -2.52345513e+06, -2.52346681e+06, -2.52347851e+06],
       [-1.78540722e+01, -2.70480097e+01, -4.04679943e+01, ...,
        -2.52345160e+06, -2.52346328e+06, -2.52347498e+06],
       [-1.75419048e+01, -2.98047319e+01, -3.65239099e+01, ...,
        -2.52344938e+06, -2.52346106e+06, -2.52347276e+06],
       ...,
       [-1.81933667e+01, -3.07136940e+01, -4.26890973e+01, ...,
        -2.52345443e+06, -2.52346613e+06, -2.52347781e+06],
       [-1.45656909e+01, -2.71311902e+01, -4.03941548e+01, ...,
        -2.52345080e+06, -2.52346250e+06, -2.52347418e+06],
       [-1.27488328e+01, -2.90773054e+01, -4.09486589e+01, ...,
        -2.52344898e+06, -2.52346068e+06, -2.52347236e+06]])

In [57]:
best_paths

array([[ 0, 14, 24, ..., 14, 14, 14],
       [ 0,  3, 35, ...,  3,  3,  3],
       [ 0, 33,  7, ...,  7,  7,  7],
       ...,
       [ 0, 31, 33, ...,  6,  6,  6],
       [ 0, 44, 39, ...,  6,  6,  6],
       [ 0,  9, 38, ...,  6,  6,  6]])

In [59]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    """
    Зворотне проходження алгоритму Вітербі.
    """
    m = best_paths.shape[1]
    z = [None] * m
    pred = [None] * m

    # Знаходження найкращого тегу для останнього слова
    best_prob_for_last_word = float('-inf')

    for k in range(best_probs.shape[0]):
        if best_probs[k, m-1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, m-1]
            z[m-1] = k

    pred[m-1] = states[z[m-1]]

    # Зворотне проходження для знаходження найкращих тегів
    for i in range(m-2, -1, -1):
        z[i] = best_paths[z[i+1], i+1]
        pred[i] = states[z[i]]

    return pred

In [60]:
pred = viterbi_backward(best_probs, best_paths, training_corpus, states)

In [61]:
pred

['NNP',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 ')',
 '.',
 '

In [65]:
def compute_accuracy(pred, y):
    """
    Обчислення точності моделі POS-тегування.
    """
    num_correct = 0
    total = 0

    for prediction, y_item in zip(pred, y):
        y_item = y_item.strip()
        word_tag_tuple = y_item.split('\t')

        if len(word_tag_tuple) != 2:
            continue

        word, tag = word_tag_tuple

        if tag == prediction:
            num_correct += 1

        total += 1

    return num_correct / total

In [62]:
with open("/content/conll_test.pos", 'r') as f:
    y = f.readlines()

In [63]:
y

['But\tCC\n',
 'the\tDT\n',
 'statistics\tNNS\n',
 'released\tVBD\n',
 'yesterday\tNN\n',
 'show\tNN\n',
 'the\tDT\n',
 'firms\tNNS\n',
 'did\tVBD\n',
 'nothing\tNN\n',
 'of\tIN\n',
 'the\tDT\n',
 'sort\tNN\n',
 '.\t.\n',
 '\n',
 'Nestle\tNNP\n',
 'Korea\tNNP\n',
 'Ltd.\tNNP\n',
 'opened\tVBD\n',
 'a\tDT\n',
 'coffee\tNN\n',
 'and\tCC\n',
 'non-dairy-creamer\tNN\n',
 'plant\tNN\n',
 'in\tIN\n',
 'Chongju\tNNP\n',
 ',\t,\n',
 'South\tNNP\n',
 'Korea\tNNP\n',
 '.\t.\n',
 '\n',
 'They\tPRP\n',
 'wear\tVBP\n',
 'tuxedos\tNNS\n',
 'most\tRBS\n',
 'nights\tNNS\n',
 ',\t,\n',
 'unless\tIN\n',
 'circumstances\tNNS\n',
 '-LRB-\t(\n',
 'a\tDT\n',
 'regular\tJJ\n',
 'gig\tNN\n',
 'at\tIN\n',
 'a\tDT\n',
 '``\t``\n',
 'tropical\tJJ\n',
 "''\t''\n",
 'lounge\tNN\n',
 ',\t,\n',
 'for\tIN\n',
 'example\tNN\n',
 '-RRB-\t)\n',
 'require\tVB\n',
 'them\tPRP\n',
 'to\tTO\n',
 'wear\tVB\n',
 'special\tJJ\n',
 'costumes\tNNS\n',
 ',\t,\n',
 'like\tIN\n',
 'Hawaiian\tNNP\n',
 'shirts\tNNS\n',
 '.\t.\n',
 '\

In [66]:
compute_accuracy(pred, y)

0.02152502596852999

In [67]:
len(pred)-len(y)

161700

In [69]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    """
    Створення матриці емісій B.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocab)

    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        for j in range(num_words):
            key = (all_tags[i], vocab[j])

            count = 0
            if key in emission_counts:
                count = emission_counts[key]

            count_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)

    return B


In [71]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

# Отримання списку всіх тегів
states = sorted(tag_counts.keys())

# Параметр згладжування
alpha = 0.001

# Створення індексованого словника слів
word_to_index = {}
for i, word in enumerate(sorted(vocab.keys())):
    word_to_index[word] = i

# Додавання спеціальних токенів
for special_token in ["--n--", "--unk--", "--unk_digit--", "--unk_punct--", "--unk_upper--",
                      "--unk_noun--", "--unk_verb--", "--unk_adj--", "--unk_adv--"]:
    if special_token not in word_to_index:
        word_to_index[special_token] = len(word_to_index)

# Створення матриць переходів та емісій
A = create_transition_matrix(alpha, tag_counts, transition_counts)
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(word_to_index.keys()))

prep = []
for line in y:
    line = line.strip()
    if line == "":
        continue
    parts = line.split('\t')
    if len(parts) >= 1:
        prep.append(parts[0])

# Ініціалізація алгоритму Вітербі
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, word_to_index)

# Пряме проходження
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, word_to_index)

# Зворотне проходження для отримання найкращої послідовності тегів
pred = viterbi_backward(best_probs, best_paths, prep, states)

# Обчислення точності
accuracy = compute_accuracy(pred, y)
print(f"Точність моделі: {accuracy:.4f}")


Точність моделі: 0.0646


In [74]:
import nltk
from nltk import word_tokenize
from nltk.corpus import treebank

In [77]:
# Завантаження необхідних ресурсів
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')
nltk.download('averaged_perceptron_tagger_eng')
# Приклад тексту
text = "The economy's temperature will be taken from several vantage points this week."

# Токенізація тексту
tokens = word_tokenize(text)

# POS-тегування з використанням NLTK
nltk_tags = nltk.pos_tag(tokens)

print("Результат POS-тегування NLTK:")
for word, tag in nltk_tags:
    print(f"{word}: {tag}")

# Оцінка точності NLTK POS-tagger на корпусі treebank
train_data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:]

# Навчання NLTK тегера
from nltk.tag import UnigramTagger, BigramTagger
unigram_tagger = UnigramTagger(train_data)
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)

# Оцінка точності
nltk_accuracy = bigram_tagger.evaluate(test_data)
print(f"Точність NLTK BigramTagger: {nltk_accuracy:.4f}")

# Порівняння з нашою реалізацією
print(f"Точність нашої реалізації: {accuracy:.4f}")
print(f"Різниця: {abs(accuracy - nltk_accuracy):.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Результат POS-тегування NLTK:
The: DT
economy: NN
's: POS
temperature: NN
will: MD
be: VB
taken: VBN
from: IN
several: JJ
vantage: NN
points: NNS
this: DT
week: NN
.: .


<ipython-input-77-0b5526ada91f>:30: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  nltk_accuracy = bigram_tagger.evaluate(test_data)


Точність NLTK BigramTagger: 0.8648
Точність нашої реалізації: 0.0646
Різниця: 0.8002
